In [1]:
from datasets import load_dataset, Image
from torch.utils.data import DataLoader
from mit_semseg.models.resnet import resnet50
from mit_semseg.models import ModelBuilder, SegmentationModule 
import PIL
import torch
import torch.nn as nn

In [2]:
def prep_dataset(sample):
    img = sample['image']
    label = sample['annotation']
    sample['width'] = img.width
    sample['height'] = img.height

    if (img.mode == 'L') | (img.mode == 'CMYK') | (img.mode == 'RGBA'):
        rgbimg = PIL.Image.new("RGB", img.size)
        rgbimg.paste(img)
        img = rgbimg

    img = img.resize((256,256),resample=PIL.Image.LANCZOS)
    label = label.resize((256,256),resample=PIL.Image.LANCZOS)
    
    sample['image'] = Image().encode_example(img)
    sample['annotation'] = Image().encode_example(label)
    return sample

In [3]:
dataset = load_dataset("scene_parse_150",split='train[0:1024]')
dataset = dataset.map(prep_dataset)
dataset.set_format("torch")
encoder = ModelBuilder().build_encoder(
    arch='resnet50dilated',
    fc_dim=2048,
    weights='')
decoder = ModelBuilder().build_decoder(
    arch='ppm',
    fc_dim=2048,
    num_class=150,
    weights='',
    use_softmax=False)
crit = nn.NLLLoss(ignore_index=-1)
sm = SegmentationModule(encoder,decoder,crit)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

/tmp/ipykernel_236240/3640807120.py:12: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((256,256),resample=PIL.Image.LANCZOS)
/tmp/ipykernel_236240/3640807120.py:13: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  label = label.resize((256,256),resample=PIL.Image.LANCZOS)
Downloading: "http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet50-imagenet.pth" to ./pretrained/resnet50-imagenet.pth


In [33]:
x = dataset[0:4]['image'].to(torch.float32)
x = x.permute((0,3,1,2))
x.shape

torch.Size([4, 3, 256, 256])

In [34]:
y = encoder.forward(x)

In [37]:
y[0].shape

torch.Size([4, 2048, 32, 32])

In [39]:
z = decoder.forward(y)

In [42]:
z[0].shape

torch.Size([150, 32, 32])

In [6]:
y.shape

torch.Size([64, 256, 256])